# Exercise 2

## Unsupervised Learning

---

## Overview

Welcome to this exercise. We are now going to use our new skills to build our first unsupervised learning models. In particular, we are going to use multiple methods to create a number unsupervisoed learning models such kmeans for clustering and PCA to reduce the dimensionality of or data.

Let's start by upgrading the `yfinance` module to the latest version. Run the following code block to upgrade it, then click the menu item at the top: **Kernel** > **Restart**, before running the remaining code blocks.

In [16]:
!pip install --upgrade yfinance

We also need to install `lxml` since we will be working with an XML document (no need to restart the kernel after running this command, since we're not upgrading).

In [18]:
!pip install lxml

In [19]:
import sys
import os
# Manually set the path relative to the py file's location that you want to import
func_lib_path = os.path.abspath(os.path.join(os.getcwd(), '../'))# Add the path to sys.path

# Add the path to sys.path
sys.path.append(func_lib_path)

# Now you can import func_lib
import func_lib
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [20]:
# Generate historical price data using the 'create_hist_prices' function from 'func_lib'.
# Assign the result to the variable 'historical_prices'.
historical_prices = func_lib.create_hist_prices()

# Define a list of momentum periods to be used in the computation.
# Include periods such as 1, 2, 3, 29, 30, 31, 59, 60, and 61.
list_of_momentums = [1, 2, 3, 29, 30, 31, 59, 60, 61]

# Compute the returns using the 'compute_returns' function from 'func_lib'.
# Pass 'historical_prices' and 'list_of_momentums' as arguments.
# Assign the result to the variable 'total_returns'.
total_returns = func_lib.compute_returns(historical_prices, list_of_momentums)

# Drop the 'F_1_d_returns' column from the 'total_returns' DataFrame.
# Use the 'drop()' method with 'columns' parameter and set 'inplace=True' to modify the DataFrame in place.
total_returns.drop(columns={'F_1_d_returns'}, inplace=True)

# Remove any rows with missing values from the 'total_returns' DataFrame.
# Use the 'dropna()' method with 'inplace=True' to modify the DataFrame in place.
total_returns.dropna(inplace=True)

[*********************100%***********************]  501 of 501 completed

1 Failed download:
['SW']: YFPricesMissingError('possibly delisted; no price data found  (1d 2000-01-01 -> 2024-05-01) (Yahoo error = "Data doesn\'t exist for startDate = 946702800, endDate = 1714536000")')


In [21]:
# Standardize the features in the 'total_returns' DataFrame.
# - Create a StandardScaler object.
# - Use the 'fit_transform()' method of the scaler to standardize the features in 'total_returns'.
# - Assign the standardized data to the variable 'X_scaled'.
scaler=StandardScaler()
X_scaled=scaler.fit_transform(total_returns)



In [ ]:
# Initialize PCA (Principal Component Analysis).
# - Create a PCA object to perform dimensionality reduction.
pca = PCA()


# Fit PCA to the standardized data.
# - Use the 'fit()' method of the PCA object to compute the principal components based on 'X_scaled'.

# Transform the data to principal components.
# - Use the 'transform()' method of the PCA object to project 'X_scaled' into the principal component space.
# - Assign the transformed data to the variable 'X_pca'.

X_pca=pca.fit_transform(X_scaled)

In [ ]:
# Compute the explained variance ratio for each principal component.
# - Use the 'explained_variance_ratio_' attribute of the PCA object to get the proportion of variance explained by each component.
# - Assign the result to the variable 'explained_variance_ratio'.
explained_variance_ratio = pca.explained_variance_ratio_
#pca.explained_variance_ratio_.sum()

# Calculate the cumulative explained variance.
# - Use 'np.cumsum()' to compute the cumulative sum of the explained variance ratios.
# - Assign the result to the variable 'cumulative_explained_variance'.
cumulative_explained_variance=np.cumsum(pca.explained_variance_ratio_)
print(cumulative_explained_variance)



In [ ]:
# Plot the cumulative explained variance.
# - Create a new figure with a specified size for the plot.
# - Plot 'cumulative_explained_variance' against the number of principal components.
# - Use markers and a dashed line style for better visualization.
# - Add x-axis and y-axis labels, a title, and a grid to the plot.
# - Display the plot using 'plt.show()'.

plt.figure(figsize=(10, 6))
plt.plot(range(1, len(cumulative_explained_variance) + 1), cumulative_explained_variance, marker='o', linestyle='--')
plt.xlabel('Number of Principal Components')
plt.ylabel('Cumulative Explained Variance')
plt.title('Cumulative Explained Variance by Principal Components')
plt.grid(True)
plt.show()

In [ ]:
# Select the number of principal components that explain at least 90% of the variance.
# - Use 'np.argmax()' to find the index of the first cumulative explained variance value that is greater than or equal to 0.90.
# - Add 1 to the index because 'np.argmax()' returns a zero-based index.
# - Assign the result to 'n_components' and print the value to display the number of components needed.
n_components = np.argmax(cumulative_explained_variance >= 0.9) + 1
print(n_components)



In [ ]:
# Apply PCA with the selected number of components.
# - Create a PCA object with 'n_components' set to the number of components that explain at least 90% of the variance.
# - Fit the PCA object to the standardized data and transform it into the principal component space.
# - Assign the transformed data to 'X_pca'.
pca=PCA(n_components=n_components)
X_pca=pca.fit_transform(X_scaled)



In [ ]:
# Define the feature names.
# - Extract the column names from the 'total_returns' DataFrame to use as feature names.
# - Assign these column names to the variable 'features'.
#print(total_returns)
features=total_returns.columns
#print(features)


# Get the loadings of the principal components.
# - Use the 'components_' attribute of the PCA object to get the principal component loadings.
# - Assign the result to the variable 'loadings'.
loadings=pca.components_.T
#print(loadings)

# Create a DataFrame of the loadings.
# - Transpose the 'loadings' array using '.T' to match the features with the principal components.
# - Use 'pd.DataFrame()' to create the DataFrame, setting the index to 'features' and the columns to principal component names.
# - Assign the DataFrame to 'loadings_df'.
loadings_df=pd.DataFrame(loadings,index=features,columns=[f'PC{i+1}' for i in range(n_components)])

# Display the loadings DataFrame.
# - Use 'print()' to output the DataFrame to the console.
print(loadings_df)
